In [ ]:
import pandas as pd

url = 'https://en.wikipedia.org/wiki/Serena_Williams'
#url = 'https://en.wikipedia.org/wiki/Venus_Williams'
#url = 'https://en.wikipedia.org/wiki/Simona_Halep'
#url = 'https://en.wikipedia.org/wiki/Maria_Sharapova'
#url = 'https://en.wikipedia.org/wiki/Caroline_Wozniacki'
#url = 'https://en.wikipedia.org/wiki/Victoria_Azarenka'
#url = 'https://en.wikipedia.org/wiki/Petra_Kvitova'
#url = 'https://en.wikipedia.org/wiki/Angelique_Kerber'
#url = 'https://en.wikipedia.org/wiki/Agnieszka_Radwanska'
#url = 'https://en.wikipedia.org/wiki/Svetlana_Kuznetsova'
#url = 'https://en.wikipedia.org/wiki/Martina_Hingis'
#url = 'https://en.wikipedia.org/wiki/Garbine_Muguruza'
#url = 'https://en.wikipedia.org/wiki/Kim_Clijsters'
#url = 'https://en.wikipedia.org/wiki/Karolina_Pliskova'
#url = 'https://en.wikipedia.org/wiki/Ashleigh_Barty'
#url = 'https://en.wikipedia.org/wiki/Lindsay_Davenport'
#url = 'https://en.wikipedia.org/wiki/Steffi_Graf'
#url = 'https://en.wikipedia.org/wiki/Martina_Navratilova'
#url = 'https://en.wikipedia.org/wiki/Elina_Svitolina'
#url = 'https://en.wikipedia.org/wiki/Naomi_Osaka'
#url = 'https://en.wikipedia.org/wiki/Justine_Henin'
#url = 'https://en.wikipedia.org/wiki/Samantha_Stosur'
#url = 'https://en.wikipedia.org/wiki/Jelena_Jankovic'

#url = 'https://en.wikipedia.org/wiki/Iga_Swiatek'
#url = 'https://en.wikipedia.org/wiki/Cori_Gauff'
#url = 'https://en.wikipedia.org/wiki/Emma_Raducanu'

In [ ]:
tables = pd.read_html(url)
print(type(tables))
print(len(tables))

In [ ]:
i = 0
while i < 5:
    table =  tables[i]
    (rows, cols) = table.shape
    if (rows < 10) | (cols < 2):
        i += 1
        continue
    else:
        print(i)
        print(table.shape)
        break
        
table

In [ ]:
if len(table.columns) == 2:
    table.columns = ['Attr', 'Value']
else:
    table.columns = ['Attr', 'Value', 'Extra']
table

In [ ]:
row_lookup = {
    'CountryWiki': 'Country',
    'Born': 'Born',
    'Height': 'Height',
    'TurnedPro': 'Turned.*pro',
    'Retired': 'Retired',
    'Coach': 'Coach',
    'EarningsWiki': 'Prize.*money',
    'CareerRecord': 'Career.*record',
    'Titles': 'Career.*titles',
    'HighRank': 'Highes.*ranking',
    }
row_lookup

The table needs to be navigated by looping through the data frame rows and the looping needs to abort once the value 'Doubles' is encountered.  Otherwise, looping by lookup of the Attr names returns dataframes if the same attribute is found multiple times.

In [ ]:
doubles_idx = table.index[table['Attr'] == 'Doubles'].to_list()
sub_tab = table[:doubles_idx[0]]
sub_tab

OK, got the proper subset of the data table (sub_tab), but still having problem with extracting the cell values.
What next?
Need the following to extract a value of the cell:  
sub_tab.loc[sub_tab.index[4], 'Value']

In [ ]:
for key in row_lookup.keys():
    print(key)
    val = row_lookup[key]
    #print(val)
    if len(sub_tab[sub_tab['Attr'].str.contains(val, regex=True, na=False)]) > 0:
        row_idx = sub_tab[sub_tab['Attr'].str.contains(val, regex=True, na=False)].index[0]
        print(sub_tab.loc[sub_tab.index[row_idx], 'Value'])
